# Vectorization

## Использование

Запустите ячейки по порядку.

In [86]:
import pandas as pd
import numpy as np
import os
import sys
import pickle
import argparse
from pathlib import Path


def find_project_root(start: Path) -> Path:
    """Ищем корень проекта, чтобы относительные пути работали и в Jupyter, и в CLI."""
    for p in [start] + list(start.parents):
        if (p / "run_pipeline.py").exists() and (p / "files").exists():
            return p
    return start


PROJECT_ROOT = find_project_root(Path.cwd())
FILES_DIR = PROJECT_ROOT / "files"

# Попытка импортировать sklearn
try:
    from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
    from sklearn.decomposition import TruncatedSVD
    import joblib
    HAS_SKLEARN = True
except ImportError:
    HAS_SKLEARN = False
    print("ОШИБКА: sklearn не установлен!")
    print("Установите: pip install scikit-learn")
    sys.exit(1)



In [ ]:
# НАСТРОЙКИ


In [88]:
# По умолчанию берём файлы из files/, но поддерживаем и запуск из других папок
_default_input = FILES_DIR / "dataset_cleaned.csv"
if not _default_input.exists():
    _default_input = PROJECT_ROOT / "dataset_cleaned.csv"

INPUT_FILE = str(_default_input)  # Файл с очищенными данными
TEXT_COLUMN = "cleaned"  # Столбец с текстом для векторизации

# Куда сохраняем результаты (по умолчанию: files/vectorized)
OUTPUT_DIR = str((FILES_DIR / "vectorized") if FILES_DIR.exists() else (PROJECT_ROOT / "vectorized"))

# Методы векторизации
VECTORIZATION_METHODS = {
    'tfidf': {
        'name': 'TF-IDF',
        'class': TfidfVectorizer,
        'params': {
            'max_features': 5000,  # Максимальное количество признаков
            'ngram_range': (1, 2),  # Униграммы и биграммы
            'min_df': 2,  # Минимальная частота документа
            'max_df': 0.95,  # Максимальная частота документа
            'lowercase': True,
            'analyzer': 'word'
        }
    },
    'count': {
        'name': 'Count Vectorizer (Bag of Words)',
        'class': CountVectorizer,
        'params': {
            'max_features': 5000,
            'ngram_range': (1, 2),
            'min_df': 2,
            'max_df': 0.95,
            'lowercase': True,
            'analyzer': 'word'
        }
    },
    'hash': {
        'name': 'Hashing Vectorizer',
        'class': HashingVectorizer,
        'params': {
            'n_features': 5000,  # Размерность вектора
            'ngram_range': (1, 2),
            'lowercase': True,
            'analyzer': 'word'
        }
    }
}



In [ ]:
# ФУНКЦИИ


In [90]:
def load_data(file_path, text_column, sample_size=None):
    """Загрузка данных из CSV файла"""
    print(f"\n📂 Загрузка данных из {file_path}...")
    
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"Файл {file_path} не найден!")
    
    if sample_size:
        df = pd.read_csv(file_path, nrows=sample_size)
        print(f"   Загружена выборка: {len(df):,} строк")
    else:
        df = pd.read_csv(file_path)
        print(f"   Загружено: {len(df):,} строк")
    
    if text_column not in df.columns:
        raise ValueError(f"Столбец '{text_column}' не найден! Доступные столбцы: {list(df.columns)}")
    
    # Проверка на пустые значения
    null_count = df[text_column].isna().sum()
    if null_count > 0:
        print(f"   ⚠️  Найдено {null_count} пустых значений, они будут удалены")
        df = df.dropna(subset=[text_column])
    
    # Фильтрация слишком коротких текстов
    df = df[df[text_column].astype(str).str.len() > 2]
    
    print(f"   ✅ Готово к векторизации: {len(df):,} строк")
    return df



In [91]:
def vectorize_text(df, text_column, method='tfidf', output_dir='vectorized'):
    """
    Векторизация текста выбранным методом
    
    Parameters:
    -----------
    df : DataFrame
        Датафрейм с текстами
    text_column : str
        Название столбца с текстом
    method : str
        Метод векторизации ('tfidf', 'count', 'hash')
    output_dir : str
        Директория для сохранения результатов
    """
    if method not in VECTORIZATION_METHODS:
        raise ValueError(f"Неизвестный метод: {method}. Доступные: {list(VECTORIZATION_METHODS.keys())}")
    
    method_info = VECTORIZATION_METHODS[method]
    print(f"\n🔢 Векторизация методом: {method_info['name']}")
    
    # Создание директории для результатов
    Path(output_dir).mkdir(exist_ok=True)
    
    # Получение текстов
    texts = df[text_column].fillna("").astype(str).tolist()
    print(f"   Обработка {len(texts):,} текстов...")
    
    # Создание векторизатора
    vectorizer_class = method_info['class']
    params = method_info['params'].copy()
    
    # Специальная обработка для HashingVectorizer
    if method == 'hash':
        vectorizer = vectorizer_class(**params)
    else:
        vectorizer = vectorizer_class(**params)
    
    # Векторизация
    print("   Выполнение векторизации...")
    vectors = vectorizer.fit_transform(texts)
    
    print(f"   ✅ Векторизация завершена!")
    print(f"   Размерность векторов: {vectors.shape}")
    print(f"   Количество признаков: {vectors.shape[1]:,}")
    
    # Сохранение векторизатора
    vectorizer_path = os.path.join(output_dir, f'vectorizer_{method}.pkl')
    joblib.dump(vectorizer, vectorizer_path)
    print(f"   💾 Векторизатор сохранён: {vectorizer_path}")
    
    # Сохранение векторов в разных форматах
    base_name = f'vectors_{method}'
    
    # 1. Sparse matrix (scipy sparse format) - самый эффективный для больших данных
    sparse_path = os.path.join(output_dir, f'{base_name}_sparse.npz')
    np.savez_compressed(sparse_path, data=vectors.data, indices=vectors.indices, 
                       indptr=vectors.indptr, shape=vectors.shape)
    print(f"   💾 Sparse матрица сохранена: {sparse_path}")
    
    # 2. Dense matrix (numpy array) - для небольших данных или когда нужен полный доступ
    if vectors.shape[1] <= 10000:  # Сохраняем dense только если признаков не слишком много
        dense_vectors = vectors.toarray()
        dense_path = os.path.join(output_dir, f'{base_name}_dense.npy')
        np.save(dense_path, dense_vectors)
        print(f"   💾 Dense матрица сохранена: {dense_path}")
        
        # 3. CSV с метаданными (первые N признаков для просмотра)
        if vectors.shape[1] <= 1000:
            df_vectors = pd.DataFrame(dense_vectors[:, :min(100, vectors.shape[1])])
            df_vectors.columns = [f'feature_{i}' for i in df_vectors.columns]
            # Добавляем исходные данные
            df_vectors['id'] = df['id'].values if 'id' in df.columns else range(len(df_vectors))
            df_vectors['text'] = df[text_column].values
            csv_path = os.path.join(output_dir, f'{base_name}_sample.csv')
            df_vectors.to_csv(csv_path, index=False, encoding='utf-8')
            print(f"   💾 Пример векторов (CSV): {csv_path}")
    
    # 4. Создание DataFrame с метаданными и ссылкой на векторы
    df_result = df.copy()
    df_result['vector_file'] = f'{base_name}_sparse.npz'
    df_result['vector_index'] = range(len(df_result))
    metadata_path = os.path.join(output_dir, f'metadata_{method}.csv')
    df_result.to_csv(metadata_path, index=False, encoding='utf-8')
    print(f"   💾 Метаданные сохранены: {metadata_path}")
    
    # Информация о признаках (для TF-IDF и Count)
    if method in ['tfidf', 'count']:
        feature_names = vectorizer.get_feature_names_out()
        feature_df = pd.DataFrame({
            'feature_index': range(len(feature_names)),
            'feature_name': feature_names
        })
        features_path = os.path.join(output_dir, f'features_{method}.csv')
        feature_df.to_csv(features_path, index=False, encoding='utf-8')
        print(f"   💾 Названия признаков сохранены: {features_path}")
    
    return vectors, vectorizer



In [92]:
def reduce_dimensions(vectors, n_components=100, method='svd'):
    """
    Уменьшение размерности векторов
    
    Parameters:
    -----------
    vectors : sparse matrix
        Векторизованные тексты
    n_components : int
        Количество компонент для уменьшения размерности
    method : str
        Метод ('svd' - TruncatedSVD)
    """
    print(f"\n📉 Уменьшение размерности до {n_components} компонент...")
    
    if method == 'svd':
        reducer = TruncatedSVD(n_components=n_components, random_state=42)
        vectors_reduced = reducer.fit_transform(vectors)
        print(f"   ✅ Размерность уменьшена: {vectors.shape} → {vectors_reduced.shape}")
        return vectors_reduced, reducer
    else:
        raise ValueError(f"Неизвестный метод уменьшения размерности: {method}")



In [93]:
def load_vectors(vector_file, metadata_file=None):
    """
    Загрузка сохранённых векторов
    
    Parameters:
    -----------
    vector_file : str
        Путь к файлу с векторами (.npz для sparse, .npy для dense)
    metadata_file : str, optional
        Путь к файлу с метаданными
    """
    from scipy.sparse import csr_matrix
    
    print(f"\n📂 Загрузка векторов из {vector_file}...")
    
    if vector_file.endswith('.npz'):
        # Sparse matrix
        loaded = np.load(vector_file, allow_pickle=True)
        vectors = csr_matrix((loaded['data'], loaded['indices'], loaded['indptr']), 
                            shape=loaded['shape'])
        print(f"   ✅ Загружено: {vectors.shape}")
    elif vector_file.endswith('.npy'):
        # Dense matrix
        vectors = np.load(vector_file)
        print(f"   ✅ Загружено: {vectors.shape}")
    else:
        raise ValueError(f"Неподдерживаемый формат файла: {vector_file}")
    
    if metadata_file and os.path.exists(metadata_file):
        metadata = pd.read_csv(metadata_file)
        print(f"   ✅ Метаданные загружены: {len(metadata)} строк")
        return vectors, metadata
    
    return vectors


In [94]:
def run_vectorization(method='tfidf', input_file=None, text_column=None, 
                      sample_size=None, reduce_dim=False, components=100, 
                      output_dir=None):
    """
    Запуск векторизации (версия для Jupyter notebook)
    
    Parameters:
    -----------
    method : str
        Метод векторизации ('tfidf', 'count', 'hash')
    input_file : str, optional
        Входной файл (по умолчанию: INPUT_FILE)
    text_column : str, optional
        Столбец с текстом (по умолчанию: TEXT_COLUMN)
    sample_size : int, optional
        Размер выборки (None для всех данных)
    reduce_dim : bool
        Уменьшить размерность векторов
    components : int
        Количество компонент при уменьшении размерности
    output_dir : str
        Директория для сохранения результатов
    """
    # Используем значения по умолчанию, если не указаны
    if input_file is None:
        input_file = INPUT_FILE
    if text_column is None:
        text_column = TEXT_COLUMN
    if output_dir is None:
        output_dir = OUTPUT_DIR
    
    print("="*60)
    print("ВЕКТОРИЗАЦИЯ ТЕКСТА")
    print("="*60)
    print(f"Метод: {VECTORIZATION_METHODS[method]['name']}")
    print(f"Входной файл: {input_file}")
    print(f"Столбец с текстом: {text_column}")
    if sample_size:
        print(f"Размер выборки: {sample_size:,}")
    else:
        print("Обработка всех данных")
    if reduce_dim:
        print(f"Уменьшение размерности: {components} компонент")
    print(f"Выходная директория: {output_dir}")
    print("="*60)
    
    # Загрузка данных
    try:
        df = load_data(input_file, text_column, sample_size=sample_size)
    except Exception as e:
        print(f"❌ Ошибка при загрузке данных: {e}")
        raise
    
    # Векторизация
    try:
        vectors, vectorizer = vectorize_text(df, text_column, method=method, output_dir=output_dir)
        
        # Опциональное уменьшение размерности
        if reduce_dim and vectors.shape[1] > components:
            vectors_reduced, reducer = reduce_dimensions(vectors, n_components=components)
            
            # Сохранение уменьшенных векторов
            reduced_path = os.path.join(output_dir, f'vectors_{method}_reduced_{components}.npy')
            np.save(reduced_path, vectors_reduced)
            print(f"   💾 Уменьшенные векторы сохранены: {reduced_path}")
            
            # Сохранение редуктора
            reducer_path = os.path.join(output_dir, f'reducer_{method}_{components}.pkl')
            joblib.dump(reducer, reducer_path)
            print(f"   💾 Редуктор сохранён: {reducer_path}")
        
        print("\n" + "="*60)
        print("✅ ВЕКТОРИЗАЦИЯ ЗАВЕРШЕНА УСПЕШНО!")
        print("="*60)
        print(f"\n📁 Результаты сохранены в директории: {output_dir}/")
        print(f"   - Векторизатор: vectorizer_{method}.pkl")
        print(f"   - Векторы (sparse): vectors_{method}_sparse.npz")
        if vectors.shape[1] <= 10000:
            print(f"   - Векторы (dense): vectors_{method}_dense.npy")
        print(f"   - Метаданные: metadata_{method}.csv")
        if method in ['tfidf', 'count']:
            print(f"   - Признаки: features_{method}.csv")
        
    except Exception as e:
        print(f"❌ Ошибка при векторизации: {e}")
        import traceback
        traceback.print_exc()
        raise


def main():
    """Основная функция для запуска из командной строки"""
    # Парсинг аргументов командной строки
    parser = argparse.ArgumentParser(
        description='Векторизация текста для машинного обучения',
        formatter_class=argparse.RawDescriptionHelpFormatter,
        epilog="""
Примеры использования:
  python vectorization.py --method tfidf
  python vectorization.py --method count --sample 10000
  python vectorization.py --method tfidf --reduce-dim --components 200
  python vectorization.py --input dataset_cleaned.csv --method hash
        """
    )
    
    parser.add_argument('--method', '-m', 
                       choices=['tfidf', 'count', 'hash'],
                       default='tfidf',
                       help='Метод векторизации (по умолчанию: tfidf)')
    
    parser.add_argument('--input', '-i',
                       default=INPUT_FILE,
                       help=f'Входной файл с данными (по умолчанию: {INPUT_FILE})')
    
    parser.add_argument('--text-column', '-t',
                       default=TEXT_COLUMN,
                       help=f'Столбец с текстом (по умолчанию: {TEXT_COLUMN})')
    
    parser.add_argument('--sample', '-s',
                       type=int,
                       default=None,
                       help='Размер выборки (None для всех данных)')
    
    parser.add_argument('--reduce-dim', '-r',
                       action='store_true',
                       help='Уменьшить размерность векторов')
    
    parser.add_argument('--components', '-c',
                       type=int,
                       default=100,
                       help='Количество компонент при уменьшении размерности (по умолчанию: 100)')
    
    parser.add_argument('--output-dir', '-o',
                       default=OUTPUT_DIR,
                       help=f'Директория для сохранения результатов (по умолчанию: {OUTPUT_DIR})')
    
    args, _ = parser.parse_known_args()
    
    # Вызываем функцию векторизации
    run_vectorization(
        method=args.method,
        input_file=args.input,
        text_column=args.text_column,
        sample_size=args.sample,
        reduce_dim=args.reduce_dim,
        components=args.components,
        output_dir=args.output_dir
    )


# Запуск main() только при запуске из командной строки (не в Jupyter)
# В Jupyter используйте функцию run_vectorization() напрямую
try:
    # Проверяем, что мы не в Jupyter/IPython
    get_ipython()
    # Если мы здесь, значит мы в Jupyter - не запускаем main()
except NameError:
    # Мы не в Jupyter - можно запускать main()
    if __name__ == "__main__":
        main()
